In [3]:
import openpyxl
import json

In [4]:
################Module For Functional Programing#########################
from functools import reduce
from itertools import chain
from itertools import groupby
from copy import deepcopy

curry = lambda f: lambda a, *args: (
    f(a, *args) if (len(args)) else lambda *args: f(a, *args)
)

filter, map, reduce = curry(filter), curry(map), curry(reduce)

go = lambda *args: reduce(lambda a, f: f(a), args)  ## 함수도 축약 가능 ##


def dictUpdate(dic1, dic2):
    dic1.update(dic2)
    return dic1


def dictsMerge(dics):
    res = reduce(dictUpdate, dics)
    return res


def dictDeleteKeys(dic, keys):
    for k in keys:
        del dic[k]
    return dic


def flat(a):
    if isinstance(a, list):
        for i in a:
            yield from flat(i)
    else:
        yield a


def grpBy(iter, key):
    f, grpKeys, grps = key, [], []
    sorted_ = sorted(iter, key=f)
    for key, grp_data in groupby(sorted_, key=f):
        grpKeys.append(key)
        grps.append(list(grp_data))
    return grps, grpKeys


#########################################################################


In [13]:
from openpyxl import load_workbook

def load_excel(file_path, sheet_name=None):
    # Load the workbook
    workbook = load_workbook(filename=file_path, data_only=True)

    # Select a specific worksheet
    if sheet_name:
        sheet = workbook[sheet_name]
    else:
        sheet = workbook.active  # Or use workbook['SheetName'] to select a specific sheet

    res = []
    # Read data from the worksheet
    for row in sheet.iter_rows(values_only=True):
        res.append(row)
        # print(row)



    # Close the workbook (not strictly necessary, but good practice)
    workbook.close()

    return res

sheet_data = load_excel("resources/통합 문서1.xlsx", "5.P Family")
sheet_data

c:\Users\HEC\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


[(None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (None,
  None,
  '[Project Family List]',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (None,
  None,
  None,
  None,
  None,
  None,
  '숨기기 열',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 ('▣',
  None,
  'No',
  'Family Name',
  'G-WM 1',
  'G-WM 2',
  'G-WM 3',
  'G-WM 4',
  'S-WM 1',
  'Item',
  'WM No.',
  'Work Master',
  'Spec.',
  'Unit',
  'Quantity',
  '표준 산출식',
  '변경 산출식',
  None),
 ('▣',
  None,
  '0.Room',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 ('▣',
  None,
  0.1,
  'Room Name ',
  Non

In [30]:
def find_hdr_row(sheet_data):
    res = go(
        sheet_data,
        filter(lambda x: "▣" in str(x)),
        filter(lambda x: "No" in str(x) and "Family Name" in str(x)),
        list,
    )
    
    return res[0]

In [31]:
find_hdr_row(sheet_data)

('▣',
 None,
 'No',
 'Family Name',
 'G-WM 1',
 'G-WM 2',
 'G-WM 3',
 'G-WM 4',
 'S-WM 1',
 'Item',
 'WM No.',
 'Work Master',
 'Spec.',
 'Unit',
 'Quantity',
 '표준 산출식',
 '변경 산출식',
 None)

In [61]:
def find_effGrp(sheet_data, hdrs=None):
    if not hdrs:
        hdrs = find_hdr_row(sheet_data)
    hdr_idx_of_FamilyName = hdrs.index("Family Name")
    
    onlyEffRows = go(
        sheet_data,
        filter(lambda x: "▶" in str(x)),
        list,
    )
    
    dics_hdrs = [
        "Item",
        "WM No.",
        "Work Master",
        "Spec.",
        "Unit",
        "표준 산출식",
        "변경 산출식",
    ]
    
    res = []
    tmp = None
    for row_idx, row_cells in enumerate(onlyEffRows):
        if "True" in str(row_cells):
            if tmp:
                res.append(tmp)
            tmp = {}
            for famType in row_cells[hdr_idx_of_FamilyName].split(","):
                tmp.update(
                    {
                    famType: []
                    }
                )
        elif "True" not in str(row_cells):
            famType_names = list(tmp.keys())
            for famType in famType_names:
                # famType_key = list(famType_dic.keys())[0]
                wmDic = {}
                for x in dics_hdrs:
                    wmDic.update({x: row_cells[hdrs.index(x)]})
                tmp[famType].append(wmDic)
            
    return res

In [62]:
effGrps = find_effGrp(sheet_data)
# effGrps[4].keys()
effGrps

[{'401 Electrical Room': [{'Item': 'Trowel',
    'WM No.': 'A04AS173-00001',
    'Work Master': 'A04AS173-00001 Finishing Work Exterior/Interior Finish Work Steel Trowel Finish        ',
    'Spec.': 0,
    'Unit': 'M2',
    '표준 산출식': '=A',
    '변경 산출식': None},
   {'Item': 'Paint-AD',
    'WM No.': 'A04AM080-00005',
    'Work Master': 'A04AM080-00005 Finishing Work Painting Work Floor Painting Anti-Dust Paint       ',
    'Spec.': 0,
    'Unit': 'M2',
    '표준 산출식': '=A',
    '변경 산출식': None},
   {'Item': 'Raised Floor',
    'WM No.': 'A04AS174-00001',
    'Work Master': 'A04AS174-00001 Finishing Work Exterior/Interior Finish Work Raised Floor Top Finish: Anti-Static Vinyl Tile       600mm x 600mm / H≤600mm',
    'Spec.': 0,
    'Unit': 'M2',
    '표준 산출식': '=A',
    '변경 산출식': None},
   {'Item': 'Tile-V',
    'WM No.': 'A04AN084-00013',
    'Work Master': 'A04AN084-00013 Finishing Work Tile Work Skirt Tile Vinyl Tile     w/ Mortar Bond Coat or Adhesive  H<150mm',
    'Spec.': 0,
    'Unit